In [1]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

In [2]:
!pip install langchainhub

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


In [5]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

1. 3개의 블로그 포스팅 본문을 Load: WebBaseLoader 활용

In [7]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=(urls),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


2. 불러온 본문을 Split (Chunking) : recursive text splitter 활용

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


3. Chunks 를 임베딩하여 Vector store 저장: openai, chroma 사용

In [11]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

4. User query = ‘agent memory’ 를 받아 관련된 chunks를 retrieve

In [53]:
retriever = vectorstore.as_retriever()
# prompt = hub.pull("rlm/rag-prompt")

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

request = "agent memory"
docs = retriever.invoke(request)
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.\n\nEach element is an observation, an event directly provided by the agent.\n- Inter-agent communication can trigger new natural language statements.\n\n\nRetrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.\n\nRecency: recent events have higher scores\nImportance: distinguish mundane from core memories. Ask LM directly.\nRelevance: based on how related it is to the current situation / query.\n\n\nReflection mechanism: synthesizes memories into higher level inferences over time and guides the agent’s future behavior. They are higher-level summaries of past events (<- note that this is a bit different from self-reflection above)'),
 Document(metadata={'source': 'https://lilianweng.gith

5. User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 )

In [16]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [66]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

for doc in docs:

  response = doc.page_content

  joke_query = f"Answer the 'relevance' (yes or no) about request to response. (yes or no)\n\n[Request]:\n{request}\n\n[Response]:\n {response}"

  parser = JsonOutputParser()

  prompt = PromptTemplate(
      template="Answer the user query.\n{format_instructions}\n{query}\n",
      input_variables=["query"],
      partial_variables={"format_instructions": parser.get_format_instructions()},
  )

  chain = prompt | llm | parser
  res = chain.invoke({"query": joke_query})

  print(res)

{'response': {'definition': {'memory_stream': 'A long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.', 'elements': 'Each element is an observation, an event directly provided by the agent.', 'inter_agent_communication': 'Can trigger new natural language statements.'}, 'retrieval_model': {'description': 'Surfaces the context to inform the agent’s behavior, according to relevance, recency, and importance.', 'factors': {'recency': 'Recent events have higher scores.', 'importance': 'Distinguish mundane from core memories. Ask LM directly.', 'relevance': 'Based on how related it is to the current situation/query.'}}, 'reflection_mechanism': {'description': 'Synthesizes memories into higher level inferences over time and guides the agent’s future behavior.', 'note': 'Higher-level summaries of past events.'}}, 'relevance': 'yes'}
{'response': 'Building agents with LLM (large language model) as its core controller is a cool c